<a href="https://colab.research.google.com/github/jamunozlab/introductory_mechanics_fall_2023/blob/main/exercises/Phys_2320_exercise_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

When an external force $\vec{F}$ is applied to an object, the object accelerates according to Newton's second law $\vec{F} = m\vec{a}$. This event, like any other in the Universe, occurs along both the space $\vec{x}$ and the time $t$ axes. When we integrate over the space coordinate, be obtain the work done by the external force on the object $W = \int \vec{F} \cdot d\vec{x}$, which is equal to the change in kinetic energy $\Delta K = \frac{1}{2}m\Delta v^2 = \frac{1}{2}m (v^2 - v_0^2)$. If the acceleration is constant along the $x$-direction, we can use the kinematic equations, of which the following looks particularly suspicious:

.

$v^2 = v_0^2 + 2 a \Delta x$

.

For example, plugging into the change in kinetic energy equation above,

.

$\Delta K = \frac{1}{2}m(v_0^2 + 2 a \Delta x - v_0^2) = \frac{1}{2}m(2 a \Delta x) = ma\Delta x$

.

Since $F = ma$,

.

$\Delta K = F \Delta x = W$

.

You can see that the conservation of energy formalism was hidding in plain sight all along, even when we were learning about the kinematic equations. What happens when we integrate over the time coordinate? Assuming that the mass of the object is constant while the force is being exerted on it:

.

$\int \vec{F} dt = m \int \vec{a} dt = m \int \frac{d\vec{v}}{dt} dt = m \int d\vec{v} = m\vec{v}|_{v_0}^{v_f} = m\Delta \vec{v} = \Delta \vec{p}$

.

The quantity $\vec{p}$ is called the [*momentum*](https://en.wikipedia.org/wiki/Momentum) and the quantity $\vec{J} = \int \vec{F} dt$ is called the [*impulse*](https://en.wikipedia.org/wiki/Impulse_(physics)). You can see that the impulse is the change in momentum.

.

Because of Newton's third law, there is an action/reaction pair force that is equal in magnitude, opposite in direction, and acting on the agent that is exerting the force on the object. The impulse of the agent will be $\vec{J_a} = \int -\vec{F} dt = -\Delta \vec{p} = -\vec{J}$. It is evident then that the momentum, just like the energy, must be a conserved quantity: any momentum that is gained by an object is at the expense of another.

.

The Universe seems to be the ultimate accountant.

## Goals

The goal of this workshop is to give you a first introduction to impulse and momentum. You will work with the molecular dynamics code that you are now familiar with using the "sombrero" potential that you analyzed in the previous workshop. In particular you will use a python function (`integrate_betweeen`) that integrates the area under the curve given by a pandas series. The series that you will integrate is that of the force versus time graph, which will give you the impulse. You will compare the impulse over different time spans that correspond to stages of the generalized oscillator.

## Instructions

1. Evaluate Jupyter cells if required.

2. Complete the exercises below.

3. Answer the questions in the space provided.

For this exercise, you will put together the main concepts that you have learned about so far in this class: kinematic equations, forces, conservation of energy, and conservation of momentum. You will simulate a mass of  m=1  kg attached to a one-dimensional spring of spring constant  k=1  N/m oscillating about its equilibrium position:

* The equilibrium position will be  x0=0  m and at time  t=0  s

* The mass will be at its equilibrium and will be moving to the right at a Velocity of  v=1  m/s.

* Since the Spring force  Fs=0  at the equilibrium position, the acceleration at  t=0  is  a = 0 m/s 2 .

* We will not consider friction, so the coefficient of kinetic friction, $\mu_k = 0.000$.

We set the initial conditions by giving appropriate values to the python variables. The potential energy in this case is given by the "sombrero" potential: $U_{\rm sombrero} = \frac{1}{2} \left[ k (\Delta \vec{x})^2 \right]^2 -  \left[ \frac{1}{2}k (\Delta \vec{x})^2 \right]$


In [1]:
# Make our lives easier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# This is the initialization cell!
position = 0.0 # position in meters
velocity = 1.0 # velocity in meters per second
acceleration = 0.0 # acceleration in meters per second^2
force = 0.0 # force in Newtons
time = 0.0 # time in seconds
spring_constant = 1.0 # force constant in kg/second^2
mass = 1.0 # mass in kg
time_step = 0.001 # time step in seconds
g = 9.8 # acceleration due to gravity m/s^2
mu = 0.00 # coefficient of kinetic friction #CHANGE THIS VARIABLE FOR EXERCISE 12.3

* The following *molecular dynamics* code uses the kinematic equations to continually update the position, velocity and; by taking the derivative of the given potential, the force exerted on our mass, and it's acceleration.

* The kinematic equations arise from integrating the equations of motion with constant acceleration, which does not occur in the case of the mass/spring system since the force (and hence the acceleration) is a function of the displacement.

* Nevertheless, the acceleration is constant if the time step we consider is infinitesimally small. A computer can approximate this by using very small time steps. For our simulation, a time step $\Delta t = 0.01$ s produces good results. The results are plotted below the code.

In [ ]:
#'sombrero' potential
potential_series = pd.Series([(1/2)*spring_constant*(x/100)**4 - (1/1)*spring_constant*(x/100)**2 for x in range(-250,250)], index=[x/100 for x in range(-250,250)])
potential_series.plot(xlabel="position (m)", ylabel="Spring potential energy (J)")

In [22]:
# This function calculates Spring Force by approximating the derrivative of potential with respect to position (dU/dx)
# You used it before in the previous workshop
def take_derivative_at(potential_series, position):
  diff = np.inf  #Infinity
  closest = 0
  for index, pos in enumerate(potential_series.index): #position at each index is enumerated
    if abs(position - pos) < diff: # this IF will always execute the first iteration
      diff = abs(position - pos)
      closest = index # Closest = 0
  rise = potential_series.iloc[closest+1] - potential_series.iloc[closest-1]  #  ΔU = - Uf - Ui
  run = potential_series.index[closest+1] - potential_series.index[closest-1] # Δx = xf - xi
  slope = rise/run # ΔU / Δx

  return slope

In [23]:
# Now that we've calculated Spring Force by approximating the derrivative of potential with respect to position (dU/dx)
# We can use Kinematics and Newtons second law to approximate the values of our remaining variables.

# Creating variable arrays
time_list = [time]
position_list = [position]
velocity_list = [velocity]
acceleration_list = [acceleration]
force_list = [force]

# Starting Loop
i=1
while i < 200000:
    time = time + time_step
    position = position + velocity*time_step + (1/2)*(acceleration*time_step**2) #Xf = xi + v(Δt)
    velocity = velocity + acceleration*time_step #Vf = Vi + a(Δt)
    spring_force = -1*take_derivative_at(potential_series=potential_series, position=position)  #Fs = -kΔx

    if velocity < 0: # moving to the left
      friction_force = mu*mass*g #friction to the right
    else: # moving to the right
      friction_force = -mu*mass*g # friction to the left
                                                             # We set the coefficient of friction to Zero, so friction will not affect
    if abs(velocity) < 0.001: # if moving extremely slowly
      friction_force = 0 # no friction because almost not moving

    force = spring_force + friction_force # Fnet = Fs + Fk
    acceleration = force/mass

# After all calculations have been computed they are then stored into our variable arrays

    time_list.append(time)
    position_list.append(position)
    velocity_list.append(velocity)
    acceleration_list.append(acceleration)
    force_list.append(force)
    i = i + 1

In [ ]:
position_series = pd.Series(position_list, index=time_list)
position_series.plot(ylabel="position (m)", xlabel="time (s)", xlim=(0, 20))

In [ ]:
velocity_series = pd.Series(velocity_list, index=time_list)
velocity_series.plot(ylabel="velocity (m/s)", xlabel="time (s)", xlim=(0, 20))

In [ ]:
acceleration_series = pd.Series(acceleration_list, index=time_list)
acceleration_series.plot(ylabel="acceleration (m/s^2)", xlabel="time (s)", xlim=(0, 20))

In [ ]:
force_series = pd.Series(force_list, index=time_list)
force_series.plot(ylabel="force (N)", xlabel="time (s)", xlim=(0,20)) # EDIT THIS IS THE CODE LINE TO FIND GLOBAL MINIMA/MAXIMA

In [ ]:
phase_series = pd.Series(velocity_series.values, index=position_series.values)
phase_series.plot(xlabel="position (m)", ylabel="velocity (m/s)", figsize=(7,7))

# Exercise 12.1

12.1.1. By visual inspection, identify the times $t$ at which there are local and global minima and maxima in `force_series`. HINT: You can modify the range of the horizontal axis by changing `xlim`. For example, `xlim=(0, 10)` will show you only the first 10 seconds.




In [11]:
## 12.1.1 Identify the times at which there are local and global minima and maxima in force_series
##
## Answer global minimum:
## Answer local minimum:
## Answer local maximum:
## Answer global maximum:

# Exercise 12.2

The impulse $J_x$ along the $x$-axis (the horizontal component of the impulse vector) following the definition in the introduction, is $\Delta p_x$. Below you can find a Python function, `integrate_between`, that will perform the integration of the force between to user-specified instants. Using your previous answer, compute the impulse between:

.
  
12.2.1. a global minimum and a global maximum

.

12.2.2. a global minimum and a local maximum.

.

12.2.3. Are they equal? Why yes or why not? Explain using physics arguments.

.

12.2.4. What are the differences and similarities between the two Python functions `take_derivative_at` and `integrate_between`?

.

## Integral of a function described by a pandas series

The python function below calculates the impulse imparted by the potential energy field on a particle moving in such potential by approximating the integral of the force with respect to time.

.

This function receives three arguments:

The `force_series` is a pandas series in which the time is given by the index and the force by the values.

The `time_start` is the value of the time where the integral starts.

The `time_end` is the value of the time where the integral ends.

If you are curious, the python function calculates the integral using the [trapezoid rule](https://en.wikipedia.org/wiki/Trapezoidal_rule) that you probably learned in your Calculus I course.

.

In [52]:
# This function calculates impulse (J) by approximating the integral of force with respect to time

  # This function takes in three arguments:
  # 1) force_series, which is a pandas series containing values representing the force at different times.
  # 2) User specified starting time.
  # 3) User specified ending time.

  # The function then calculates the Integral of the Force between the chosen start and end times:

    # The Central DIfference Method & "Rise Times Run Approximation": (Practical)
      #   1) Iterates through the index of the force_series and finds the "start and end times" that are closest to the user specified times. (One for loop for each)
      #   2) In each iteration, the difference between the given time and the closest time is then stored in the variable (diff).
        #   (diff) is defined as the difference between values at two times in the series that are "closest" to the the given "start and end times"
      #   3) The closest "start and end times" are then stored in the variables "closest_start & closest_end"
      #   4) Finally, we are able to approximate the integral of our force by multiplying "base * height" and returning that value

def integrate_betweeen(force_series, time_start, time_end):
  # Find the index of the series that is closest to the start time
  diff = np.inf
  closest_start = 0
  for index, time in enumerate(force_series.index): # time at each index is enumerated

    if abs(time_start - time) < diff: # This "if" will always execute the first iteration, and then only execute afterward "if" a closer position is found in the series
      diff = abs(time_start - time) # Assign "current smallest difference"
      closest_start = index # Assign closest time (smallest diffrence) to "closest_start"

  # Find the index of the series that is closest to the end time
  diff = np.inf
  closest_end = 0
  for index, time in enumerate(force_series.index): # time at each index is enumerated
    if abs(time_end - time) < diff: # This "if" will always execute the first iteration, and then only execute afterward "if" a closer position is found in the series
      diff = abs(time_end - time) # Assign "current smallest difference"
      closest_end = index # Assign closest time (smallest diffrence) to "end"

  integral = 0
  base = force_series.index[1] - force_series.index[0]
  for i in range(closest_start+1, closest_end):
    height = (force_series.iloc[i-1] + force_series.iloc[i])/2
    integral = integral + base * height

  return integral # return approximated integral

In [ ]:
# Usage example
integrate_betweeen(force_series, time_start=1.4, time_end=4.3)

In [15]:
## 12.2.1 Use the function integrate_between() to compute the impulse between a global minimum and a global maximum
##
## Answer:

In [17]:
## 12.2.2 Use the function integrate_between() to compute the impulse between a global minimum and a local maximum
##
## Answer:

In [18]:
## 12.2.3 Are the numbers in your two answers above the same? If yes, why yes? If no, why not? Use physics arguments to explain.
##
## Answer:

In [19]:
## 12.2.4 What are the differences and similarities between the two Python functions take_derivative_at() and integrate_between()?
##
## Answer:

# Exercise 12.3

In the initialization cell of the molecular dynamics code simulation, change the value of the coefficient of friction so that $mu=0.002$ and run the code again. HINTS: To be safe, restart the session runtime, and re-evaluate all code cells after making the changes. Change `xlim=(0, 200)` for the position, velocity, acceleration, and force series to visualize these quantities for the whole simulation time.

.

12.3.1. Explain the similarities and differences of position, velocity, acceleration, force, and phase when friction is included in the simulation.

.

12.3.2. Is mechanical energy conserved in the system when there is friction?

.

12.3.3. Change the initial parameters `position`, `velocity`, `acceleration` in the initialization of the dynamics simulation until you find a combination of parameters in which the object is finally at rest on the positive side of the hump in the sombrero potential.

.

In [ ]:
## 12.3.1 Explain the similarities and differences of position, velocity, acceleration, force, and phase when friction is included in the simulation.
##
## Answer position:
## Answer velocity:
## Answer acceleration:
## Answer force:
## Answer phase:

In [ ]:
## 12.3.2 Is mechanical energy conserved in the system when there is friction?
##
## Answer:

In [ ]:
## 12.3.3 Which combination of parameters results in the object stopping on the right well of the potential?
##
## Answer: